In [1]:
import tensorflow as tf


In [2]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPool2D
import os


In [3]:
from keras import layers
from keras import models

In [4]:
num_classes = 5
img_rows, img_cols = 48,48
batch_size=8

In [5]:
train_data_dir = r'C:/Users/melod/Facial_Expression_Recognition/train'
validation_data_dir=r'C:/Users/melod/Facial_Expression_Recognition/validation'

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=30,
                                  shear_range=0.3,
                                  zoom_range=0.1,
                                  width_shift_range=0.4, 
                                  height_shift_range= 0.4,
                                  horizontal_flip=True, 
                                  vertical_flip= True,
                                  fill_mode = 'nearest'
                                  )

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                   color_mode='grayscale',
                                                    target_size=(img_rows, img_cols),
                                                    batch_size= batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True
                                                   )
validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                   color_mode='grayscale',
                                                    target_size=(img_rows, img_cols),
                                                    batch_size= batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True
                                                   )

Found 24282 images belonging to 5 classes.
Found 5937 images belonging to 5 classes.


In [7]:
model = Sequential()

# VGG 사용
# block1
model.add(Conv2D(32,(3,3), 
         padding='same', kernel_initializer='he_normal',
         input_shape=(48, 48,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3), 
         padding='same', kernel_initializer='he_normal',
         input_shape=(48, 48,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# block2
model.add(Conv2D(64,(3,3), 
         padding='same', kernel_initializer='he_normal',
         input_shape=(48, 48,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3), 
         padding='same', kernel_initializer='he_normal',
         input_shape=(48, 48,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# block3

model.add(Conv2D(128,(3,3), 
         padding='same', kernel_initializer='he_normal',
         input_shape=(48, 48,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3), 
         padding='same', kernel_initializer='he_normal',
         input_shape=(48, 48,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# block4

model.add(Conv2D(256,(3,3), 
         padding='same', kernel_initializer='he_normal',
         input_shape=(48, 48,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3), 
         padding='same', kernel_initializer='he_normal',
         input_shape=(48, 48,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# block5
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

#block6
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# block7

model.add(Dense(num_classes, kernel_initializer='he_normal'))
model.add(Activation('softmax'))


In [8]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        320       
                                                                 
 activation (Activation)     (None, 48, 48, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 48, 48, 32)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 48, 48, 32)       128       
 hNormalization)                                        

In [9]:
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint('C:/Users/melod/Facial_Expression_Recognition/Emotion_little_vgg.h5',
                             monitor='val_loss',
                             mode = 'min',
                             save_best_only=True,
                             verbose = 1                             
                            )

earlystop = EarlyStopping(monitor='val_loss',
                         min_delta=0,
                         patience=3,
                         verbose=1,
                         restore_best_weights= True
                         )

reduce_lr = ReduceLROnPlateau(monitor='val_boss',
                             factor=0.2,
                             patience=3,
                             verbose=1,
                             min_delta=0.0001)

# lerning 레이트 알기 위해서 사용 
callbacks = [earlystop, checkpoint, reduce_lr]


In [10]:

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])



In [11]:
nb_train_samples = 24176
nb_validation_samples = 3006
epochs = 25

In [12]:
# fit the modelS
history = model.fit_generator(
          train_generator,
          epochs=epochs,
          steps_per_epoch=len(train_generator),
          validation_data=validation_generator,
          validation_steps=len(validation_generator),
          callbacks = call
        
         )

Epoch 1/25


C:\Users\melod\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


3036/3036 [==============================] - 210s 69ms/step - loss: 1.7063 - accuracy: 0.2555 - val_loss: 1.5358 - val_accuracy: 0.3244
Epoch 2/25
3036/3036 [==============================] - 207s 68ms/step - loss: 1.5614 - accuracy: 0.2850 - val_loss: 1.5299 - val_accuracy: 0.3308
Epoch 3/25
3036/3036 [==============================] - 208s 69ms/step - loss: 1.5406 - accuracy: 0.3003 - val_loss: 1.5282 - val_accuracy: 0.3352
Epoch 4/25
3036/3036 [==============================] - 207s 68ms/step - loss: 1.5165 - accuracy: 0.3154 - val_loss: 1.4830 - val_accuracy: 0.3436
Epoch 5/25
3036/3036 [==============================] - 208s 68ms/step - loss: 1.5022 - accuracy: 0.3250 - val_loss: 1.4290 - val_accuracy: 0.3734
Epoch 6/25
3036/3036 [==============================] - 211s 69ms/step - loss: 1.4953 - accuracy: 0.3309 - val_loss: 1.5139 - val_accuracy: 0.3510
Epoch 7/25
3036/3036 [==============================] - 215s 71ms/step - loss: 1.4690 - accuracy: 0.3504 - val_loss: 1.4674 - val

In [14]:
model.save('./model.h5')
 
print("Saved model to disk") 

Saved model to disk


In [19]:
model.save('C:/Users/melod/Facial_Expression_Recognition/facefeatures_new_model2.h5')

In [2]:
import maplotlib.pyplot as plt

ModuleNotFoundError: No module named 'maplotlib'